In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from functools import partial


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
class Flow_Decomposition(object):
    def __init__(self, H, W):
        
        self.lvl = 4
        self.filters = [16, 32, 64, 96]
        self.s_range = 4
        self.H = H    #height = 21
        self.W = W    #width = 28
        
    def FeaturePyramidExtractor(self, x):
        with tf.variable_scope("FeaturePyramidExtractor", reuse=tf.AUTO_REUSE):
            for l in range(self.lvl):
                x = tf.layers.Conv2D(self.filters[l], (3,3), (2, 2), padding = 'same')(x)
                x = tf.nn.leaky_relu(x, 0.1)
                x = tf.layers.Conv2D(self.filters[l], (3,3), (1, 1), padding = 'same')(x)
                x = tf.nn.leaky_relu(x, 0.1)
            return x
    def CostVolumeLayer(self, features_0, features_0from1):
        cost_length = (2 * self.s_range + 1) ** 2    #81

        def get_cost(features_0, features_0from1, shift):
            def pad2d(x, vpad, hpad):
                return tf.pad(x, [[0, 0], vpad, hpad, [0, 0]])

            def crop2d(x, vcrop, hcrop):
                return tf.keras.layers.Cropping2D([vcrop, hcrop])(x)

            """
            Calculate cost volume for specific shift
            - inputs
            features_0 (batch, h, w, nch): feature maps at time slice 0
            features_0from1 (batch, h, w, nch): feature maps at time slice 0 warped from 1
            shift (2): spatial (vertical and horizontal) shift to be considered
            - output
            cost (batch, h, w): cost volume map for the given shift
            """
            v, h = shift  # vertical/horizontal element
            vt, vb, hl, hr = max(v, 0), abs(min(v, 0)), max(h, 0), abs(min(h, 0))  # top/bottom left/right
            f_0_pad = pad2d(features_0, [vt, vb], [hl, hr])
            f_0from1_pad = pad2d(features_0from1, [vb, vt], [hr, hl])
            cost_pad = f_0_pad * f_0from1_pad
            return tf.reduce_mean(crop2d(cost_pad, [vt, vb], [hl, hr]), axis=3)

        get_c = partial(get_cost, features_0, features_0from1)
        cv = [0] * cost_length
        depth = 0
        for v in range(-self.s_range, self.s_range + 1):
            for h in range(-self.s_range, self.s_range + 1):
                cv[depth] = get_c(shift=[v, h])
                depth += 1

        cv = tf.stack(cv, axis=3)
        cv = tf.nn.leaky_relu(cv, 0.1)
        return cv

    def TranslationEstimator(self, feature_2, feature_0):
        def _conv_block(filters, kernel_size=(3, 3), strides=(1, 1)):
            def f(x):
                x = tf.layers.Conv2D(filters, kernel_size, strides, 'valid')(x)
                x = tf.nn.leaky_relu(x, 0.2)
                return x

            return f

        with tf.variable_scope("TranslationEstimator", reuse=tf.AUTO_REUSE):
            cost = self.CostVolumeLayer(feature_2, feature_0)
            x = tf.concat([feature_2, cost], axis=3)
            x = _conv_block(128, (3, 3), (1, 1))(x)
            x = _conv_block(128, (3, 3), (1, 1))(x)
            x = _conv_block(96, (3, 3), (1, 1))(x)
            x = _conv_block(64, (3, 3), (1, 1))(x)
            feature = _conv_block(32, (3, 3), (1, 1))(x)
            x = tf.reduce_mean(feature, axis=[1, 2])
            flow1 = tf.layers.Dense(2)(x)
            flow2 = tf.layers.Dense(2)(x)
            flow1 = tf.expand_dims(tf.expand_dims(flow1, 1), 1)
            flow2 = tf.expand_dims(tf.expand_dims(flow2, 1), 1)
            flow1 = tf.tile(flow1, [1, self.H, self.W, 1])
            flow2 = tf.tile(flow2, [1, self.H, self.W, 1])
            return flow1, flow2
    
    def build_model(self, Image):

        """P"""
        feature = []
        for i in range(5):
            feature.append(self.FeaturePyramidExtractor(Image[i]))
        print(feature)
        
        
        FF, FB = [], []
        for i in range(5):
            for j in range(5):
                if i == j :
                    continue
                else :
                    F, B = self.TranslationEstimator(feature[i], feature[j])    #initial(0,1)
                    FF.append(F)
                    FB.append(B)

        
        return FF+FB
